### 비지도 학습
- 훈련 데이터에 타겟값(Target Value)이 주어지지 않은 상태에서 학습을 수행하는 방식
- 훈련 데이터를 학습하여 모델을 생성하면서 유사한 특성(관계, 패턴 등)을 가지는 데이터를 클러스터로 구성
- 새로운 데이터의 특성을 분석하여 해당하는 클러스터를 예측

### 군집화
- 데이터를 클러스터(군집)로 구성하는 작업

### K-Means (K-평균) 알고리즘
- K개의 중심점을 임의의 위치를 기준으로 가까이에 있는 데이터를 확인한 뒤, 그룹과의 거리(유클리디안 거리의 제곱을 사용하여 계산)의 평균 지점으로 중심점을 이동하는 방식
- 가장 많이 활용하는 군집화 알고리즘이지만, 클러스터의 수를 나타내는 k를 직접 지정해야하는 문제가 있음

### 엘보 방법(elbow method)
- 왜곡(distortion): 클러스터의 중심점과 클러스터 내의 데이터 거리의 차이의 제곱값의 합
- 클러스터 개수 k의 변화에 따른 왜곡의 변화를 그래프로 그려보면 그래프가 꺽이는 지점인 엘보가 나타나는데, 그 지점의 k를 최적의 k로 선택

### 실루엣 분석(silhouette analysis)
- 클러스터 내에 있는 데이터가 얼마나 조밀하게 모여있는지를 측정하는 그래프 도구
- 데이터 i가 해당 클러스터 내의 데이터와 얼마나 가까운가를 나타내는 클러스터 응집력(cluster coherence) a(i)
- 가장 가까운 다른 클러스터 내의 데이터와 얼마나 떨어졌는가를 나타내는 클러스터 분리도(cluster separation) b(i)
- 실루엣 계수 s(i)를 계산  
    » -1에서 1사이의 값을 가지면 1에 가까울수록 좋은 군집화를 의미

## 목표: K-평균으로 온라인 판매 데이터를 분석한 후 타깃 마케팅을 위한 소비자 군집을 만듬

- [UCI Machine Learning Repository](http://archive.ics.uci.edu/)에 접속하여 ‘online retail’을 검색 & Download

In [ ]:
import pandas as pd
import math

In [ ]:
retail_df = pd.read_excel('./dataSet/Online_Retail.xlsx')
retail_df.head()

In [ ]:
# !importError 오류 발생시
# !pip install openpyxl

In [ ]:
retail_df.info()

In [ ]:
#오류 데이터 정제
retail_df = retail_df[retail_df['Quantity']>0]
retail_df = retail_df[retail_df['UnitPrice']>0]
retail_df = retail_df[retail_df['CustomerID'].notnull()]
#'CustomerID' 자료형을 정수형으로 변환
retail_df['CustomerID'] = retail_df['CustomerID'].astype(int)

In [ ]:
retail_df.info()
print(retail_df.isnull().sum())
print(retail_df.shape)

In [ ]:
#중복 레코드 제거
retail_df.drop_duplicates(inplace = True)
print(retail_df.shape) #작업 확인용 출력

In [ ]:
pd.DataFrame([{'Product':len(retail_df['StockCode'].value_counts()), 
               'Transaction':len(retail_df['InvoiceNo'].value_counts()), 
               'Customer':len(retail_df['CustomerID'].value_counts())}], columns = ['Product', 'Transaction', 
                                                                                    'Customer'], index = ['counts'])

In [ ]:
retail_df['Country'].value_counts()

In [ ]:
#주문 금액 컬럼 추가
retail_df['SaleAmount'] = retail_df['UnitPrice']*retail_df['Quantity']
retail_df.head() #작업 확인용 출력

In [ ]:
aggregations = {
    'InvoiceNo':'count',
    'SaleAmount':'sum',
    'InvoiceDate':'max'
}
customer_df = retail_df.groupby('CustomerID').agg(aggregations)
customer_df = customer_df.reset_index()
customer_df.head() #작업 확인용 출력

In [ ]:
customer_df = customer_df.rename(columns = {'InvoiceNo':'Freq', 'InvoiceDate':'ElapsedDays'})
customer_df.head() #작업 확인용 출력

In [ ]:
import datetime
# '기준 날짜 - 마지막 구매일'로 계산해 구함(날짜기준: 2011년 12월 10일)
customer_df['ElapsedDays'] = datetime.datetime(2011,12,10) - customer_df['ElapsedDays']
customer_df.head() #작업 확인용 출력

In [ ]:
customer_df['ElapsedDays'] = customer_df['ElapsedDays'].apply(lambda x: x.days+1)
customer_df.head() #작업 확인용 출력

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq'], customer_df['SaleAmount'], customer_df['ElapsedDays']], sym = 'bo')
plt.xticks([1, 2, 3], ['Freq', 'SaleAmount', 'ElapsedDays'])
plt.show()

In [ ]:
import numpy as np

customer_df['Freq_log'] = np.log1p(customer_df['Freq'])
customer_df['SaleAmount_log'] = np.log1p(customer_df['SaleAmount'])
customer_df['ElapsedDays_log'] = np.log1p(customer_df['ElapsedDays'])
customer_df.head() #작업 확인용 출력

In [ ]:
fig, ax = plt.subplots()
ax.boxplot([customer_df['Freq_log'], customer_df['SaleAmount_log'], customer_df['ElapsedDays_log']], sym = 'bo')
plt.xticks([1, 2, 3], ['Freq_log', 'SaleAmount_log', 'ElapsedDays_log'])
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

In [ ]:
X_features = customer_df[['Freq_log', 'SaleAmount_log', 'ElapsedDays_log']].values

In [ ]:
from sklearn.preprocessing import StandardScaler
X_features_scaled = StandardScaler().fit_transform(X_features)

In [ ]:
distortions = []

for i in range(2, 11):
    kmeans_i = KMeans(n_clusters = i, random_state = 0) #모델 생성
    kmeans_i.fit(X_features_scaled) #모델 훈련
    distortions.append(kmeans_i.inertia_)

plt.plot(range(2,11), distortions, marker = 'o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0) #모델 생성
#모델 학습과 결과 예측(클러스터 레이블 생성)
Y_labels = kmeans.fit_predict(X_features_scaled)

In [ ]:
customer_df['ClusterLabel'] = Y_labels
customer_df.head() 

In [ ]:
from matplotlib import cm

def silhouetteViz(n_cluster, X_features):
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0)
    Y_labels = kmeans.fit_predict(X_features)
    
    silhouette_values = silhouette_samples(X_features, Y_labels, metric = 'euclidean')
    
    y_ax_lower, y_ax_upper = 0, 0
    y_ticks = []
    
    for c in range(n_cluster):
        c_silhouettes = silhouette_values[Y_labels == c]
        c_silhouettes.sort()
        y_ax_upper += len(c_silhouettes)
        color = cm.jet(float(c) / n_cluster)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouettes, height = 1.0, edgecolor = 'none', color = color)
        y_ticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouettes)
        
    silhouette_avg = np.mean(silhouette_values)
    plt.axvline(silhouette_avg, color = 'red', linestyle = '--')
    plt.title('Number of Cluster : ' + str(n_cluster) + '\n' + 'Silhouette Score : '+ str(round(silhouette_avg, 3)))
    plt.yticks(y_ticks, range(n_cluster))
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.tight_layout()
    plt.show()

In [ ]:
def clusterScatter(n_cluster, X_features):
    c_colors = []
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0)
    Y_labels = kmeans.fit_predict(X_features)
    
    for i in range(n_cluster):
        c_color = cm.jet(float(i) / n_cluster) #클러스터의 색상 설정
        c_colors.append(c_color)
        #클러스터의 데이터 분포를 동그라미로 시각화
        plt.scatter(X_features[Y_labels == i,0], X_features[Y_labels == i,1], marker = 'o', color = c_color, edgecolor = 'black', s = 50, label = 'cluster '+ str(i))

    #각 클러스터의 중심점을 삼각형으로 표시
    for i in range(n_cluster):
        plt.scatter(kmeans.cluster_centers_[i,0], kmeans.cluster_centers_[i,1], marker = '^', color = c_colors[i], edgecolor = 'w', s = 200)

    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
silhouetteViz(2, X_features_scaled)

In [ ]:
silhouetteViz(3, X_features_scaled)

In [ ]:
silhouetteViz(4, X_features_scaled)

In [ ]:
silhouetteViz(5, X_features_scaled)

In [ ]:
silhouetteViz(6, X_features_scaled)

In [ ]:
clusterScatter(2, X_features_scaled)

In [ ]:
clusterScatter(3, X_features_scaled)

In [ ]:
clusterScatter(4, X_features_scaled)

In [ ]:
clusterScatter(5, X_features_scaled)

In [ ]:
clusterScatter(6, X_features_scaled)

In [ ]:
best_cluster = 4
kmeans = KMeans(n_clusters = best_cluster, random_state = 0)
Y_labels = kmeans.fit_predict(X_features_scaled)

In [ ]:
customer_df['ClusterLabel'] = Y_labels
customer_df.head()

In [ ]:
customer_df.to_csv('./dataSet/Online_Retail_Customer_Cluster.csv') 

In [ ]:
customer_df.groupby('ClusterLabel')['CustomerID'].count()

In [ ]:
customer_cluster_df = customer_df.drop(['Freq_log', 'SaleAmount_log', 'ElapsedDays_log'], axis = 1, inplace = False)

In [ ]:
#주문 1회당 평균 구매금액: SaleAmountAvg
customer_cluster_df['SaleAmountAvg'] = customer_cluster_df['SaleAmount']/customer_cluster_df['Freq']
customer_cluster_df.head()

In [ ]:
customer_cluster_df.drop(['CustomerID'], axis = 1, inplace = False).groupby('ClusterLabel').mean()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

def clusterScatter_3D(n_cluster, X_features):
    c_colors = []
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0)
    Y_labels = kmeans.fit_predict(X_features)

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(projection='3d')

    for i in range(n_cluster):
        c_color = cm.jet(float(i) / n_cluster) #클러스터의 색상 설정
        c_colors.append(c_color)
        #클러스터의 데이터 분포를 동그라미로 시각화
        ax.scatter(X_features[Y_labels == i,0], X_features[Y_labels == i,1], X_features[Y_labels == i,2], marker = 'o', color = c_color, edgecolor = 'black', s = 50, label = 'cluster '+ str(i))

    #각 클러스터의 중심점을 삼각형으로 표시
    for i in range(n_cluster):
        ax.scatter(kmeans.cluster_centers_[i,0], kmeans.cluster_centers_[i,1], kmeans.cluster_centers_[i,2], marker = '^', color = c_colors[i], edgecolor = 'w', s = 200)

    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
clusterScatter_3D(2, X_features_scaled)

In [ ]:
clusterScatter_3D(3, X_features_scaled)

In [ ]:
clusterScatter_3D(4, X_features_scaled)

In [ ]:
clusterScatter_3D(5, X_features_scaled)

In [ ]:
clusterScatter_3D(6, X_features_scaled)